In [1]:
from wallet import Wallet
from accounts import SHMAccount
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction

### Wallets


- CorporateSuperWallet is the main wallet. I AM THINKING TO MAKE MAIN WALLET A COMPANY WALLET INSTEAD, WHERE THERE IS NO PRIVATE KEYS.
- CorporateWallet: cosigners at the top corporate level.
- StoreWallet: responsible for only address generation for payments in each store.
- DailySafeWallet: Multisignature single-herarchical wallet. Manageble (see balance and create transactions) by everybody but AssistantManagerWallet who only can authorize in emergency events.
- WeeklySafeWallet: same as DailySafeWallet with the exception that no AssistantManagerWallet is part of this wallet.
- CorporateSafeWallet: Multisignature fully-herarchical wallet. Supported on the non_hardened keys within tge year account which is a hardened derived account.
- ManagerWallet: takes part in the DailySafeWallet and the WeeklySafeWallet.
- AssistantManagerWallet: only a cosigner wallet. This is the most basic and simple wallet type.


In [2]:
class CorporateSuperWallet(Wallet):
    
    def create_store_account(self, store_code):
        """
        Returns the master private key of the store with index 'store_code'.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        return 
    
    def request_pubkeys_daily_safe(self,store_code, m=2,n=6):
        """
        Returns the message to start requesting the public keys necessary to setup a
        daily safe.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        #save in the database the unique_id, public keys, m, n, and the store_code.
        return 
    
    def create_daily_safe_wallet(self, public_key_list, m=2, n=6,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc)
        shm_account = SHMAccount(m,n, str(daily_safes_acc.xtended_public_key), public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_weekly_safe_wallet(self,master_pubkey, public_key_list, m=2, n=5,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        path = "m/44H/0H/2H"
        weekly_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(m,n, weekly_safes_acc.xtended_public_key, public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_corporate_safe(self, index):
        return 
    
    def get_store_deposit_account(self, store_index):
        return

In [3]:
class StoreWallet(Wallet):
    
     def create_daily_safe(self, date_index, public_key_list,m=2,n=6):
        """
        returns a multisignature account with the public keys and the index specified.
        date_index: Integer. Must follow YYMMDD format.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length.
        """
        return 
    

In [4]:

class ManagerWallet(Wallet):
    
    def get_daily_safe_pubkey(self):
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        return daily_safes_acc.private_key_obj.point.sec()
    
    def accept_invite(self,invite, name):
        """
        accepts the invitation from a SuperCorporateWallet to participate in a daily-safe wallet.
        The acceptance must be through a private key that matches one of the public keys that are
        part of the SHM wallet passed in the invite.
        invite: SHMAccount object. The wallet that we are going to be part of.
        """
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc.private_key)
        
        public_key_list = invite["public_key_list"]
        master_pubkey = invite["master_pubkey"]
        m = invite["m"]
        n = invite["n"]
        addr_type = invite["addr_type"]
        segwit = invite["segwit"]
        testnet = invite["testnet"]
        
        safe_wallet = SHDSafeWallet.from_privkey_masterpubkey( name, public_key_list, master_pubkey, 
                                                              daily_safes_acc.private_key,m, n, addr_type, testnet,
                                                              segwit)
        """
        
        shm_account = SHMAccount(invite.m,invite.n, str(invite.xtended_pubkey), invite.public_keys, 
                                 daily_safes_acc.private_key_obj,invite.addr_type,invite.testnet)"""
        return safe_wallet
        

In [5]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


public_keys = [ manager1.get_daily_safe_pubkey(), manager2.get_daily_safe_pubkey(), PrivateKey(int.from_bytes(b"Ashley Serna","little")).point.sec()]


8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799


In [6]:
corp_wallet=CorporateSuperWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'other seek spend wise milk govern conduct crunch train wrestle twist cattle'
7349618075688827342644292161792411154388397071333136974711210419399617419860687939173705085036785308619307965881391603570352784472532296452988784284606559


In [7]:
invite = corp_wallet.create_daily_safe_wallet(public_keys,n=4,testnet=True)

tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda


In [8]:
daily_wallet_manager1 = manager1.accept_daily_safe_wallet(invite)

b'\xd3\xcdG\xa0\xa1\xbf\x86V\x879\xa2\x05Q3o\xb7(\x92Z\x19\xc8bzf\xf7\x9e\xd4\xfb\xc9\xf5\xbdP'


In [9]:
daily_wallet_manager2 = manager2.accept_daily_safe_wallet(invite)

b"\xfe\x8e\xa2X\xe1\xe0\x9b\xef\xd2s\x1f;\xd5\xf3g\x88\xfd\x97\x13C/\xe3H\xdf\xb8E\xcc[$H\x1f'"


In [15]:
deposit_m1 = daily_wallet_manager1.get_deposit_account(200820)
deposit_m1.address

'tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk'

In [16]:
deposit_m2 = daily_wallet_manager2.get_deposit_account(200820)
deposit_m2.address

'tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk'

'tb1qsklg6yhka9ea46py7rk08hnkv97685tn9gjtezv2e6tudyfc7qgsxhlypz'

'tb1qsklg6yhka9ea46py7rk08hnkv97685tn9gjtezv2e6tudyfc7qgsxhlypz'

In [18]:
prev_tx_id_list = ["d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c"]
my_tx = MultiSigTransaction.create(prev_tx_id_list, [("mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8",777)], deposit_m1, fee=None, 
                 segwit=True)

trying P2WSH
FIRST CASE: 0
transaction IS SEGWIT
transaction passed: d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c
Address: tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk
trying P2WSH
tx: 05202f76c09c782ac576e12a9756b721edcc0ce7ade978e0924b693f295717a8
version: 1
tx_ins:
d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c:1
tx_outs:
777:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
1000000:OP_0 6a80f9bcd842c857a4820d51ebbf2dca22afe6738374029d855d4fb6f05af526
locktime: 0
fee: 4096
change 95127
total 100000, diff: 0
cmds is empty. Creating new set of commands
sign_input_multisig commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x19Q\x1e"h\x82e\xe9N\x06_Q\x82TB(\xdf\xd4\xb8\xd7\xef,\xf7B\x16Z\xfeE\x01', 0, 0, 0, b"R!\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca!\x02\x17n\x

In [19]:
my_tx_signed_by2 = MultiSigTransaction.sign_received_tx(my_tx,deposit_m2)

sign_input_multisig commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x19Q\x1e"h\x82e\xe9N\x06_Q\x82TB(\xdf\xd4\xb8\xd7\xef,\xf7B\x16Z\xfeE\x01', b'0E\x02!\x00\xec\x93\xaa5\xab\xab\x8e?\xf1\xb3c\xfa\x9b\xf2c\xf1>s\x8e\xe2\x89\xe1\xb9:]\x10\x8cgL\x8dz\xba\x02 e\xa4\x8cn\x17\xe2\xe4\xa0\x82\xaf\x80\xf3d\xc0\xda\xf5\xd8\xde/}\xe1\xb9\x1cj\x02A\xb8P\x10\r\xf2T\x01', 0, 0, b"R!\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca!\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}!\x03\x03\xb0\x94-\xbe\xd2\xd0Nq\xab+\xa9\xbc\x05\xb9 \xf8O\xfe+\x8b\x8c-\x8c\xfd\x80*\xda`\xf6\xe8\xd1!\x03\x87\x8c5:r\x19J\xbb3\xab\xe3\xff\x82\xcfNQ\xdb\xefy{[\xbc\xac\x16+\x8b\xec\x1c)\x98h\x11T\xae"]
segwit commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x1

In [20]:
my_tx_signed_by2.serialize().hex()

'010000000001011c2f869e494b5c97aaba32427ba7c4fa2069eead370f7c79ebb2d2a179b929d60100000000ffffffff0209030000000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac97730100000000002200206a80f9bcd842c857a4820d51ebbf2dca22afe6738374029d855d4fb6f05af526040047304402205bdc5aa1eb9c01bbd8f5f0e77ab58ede70be9f5320dd5f9161b758d36087e6ce02207b2161fc19511e22688265e94e065f5182544228dfd4b8d7ef2cf742165afe4501483045022100ec93aa35abab8e3ff1b363fa9bf263f13e738ee289e1b93a5d108c674c8d7aba022065a48c6e17e2e4a082af80f364c0daf5d8de2f7de1b91c6a0241b850100df254018b522103fbb29f28e23a8d07f0583638fa89e865bad02dd1796bfa65ad95a517762ac2ca2102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d210303b0942dbed2d04e71ab2ba9bc05b920f84ffe2b8b8c2d8cfd802ada60f6e8d12103878c353a72194abb33abe3ff82cf4e51dbef797b5bbcac162b8bec1c2998681154ae00000000'

In [5]:
from constants import *

In [15]:
from wallet import Wallet
from accounts import SHMAccount
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction
import datetime
from bip32 import Xtended_pubkey, Xtended_privkey

class SHDSafeWallet(Wallet):
    """
    SafeWallet is a Multisignature SHD wallet.
    SHDM Single Herarchical Deterministic Multi-Signature Wallet
    """
    def __init__(self, name, public_key_list, m=2, n=6,addr_type="p2wsh",_privkey=None, master_pubkey=None, 
                 master_privkey=None, testnet=False, segwit=True, parent_name=None, safe_index=-1):
        """
        public_key_list: List of public keys. Must be in bytes.
        m: integer
        n: integer
        _privkey: PrivateKey object. If you provide a Private key, don't provide a master_privkey.
        master_pubkey: Xtended_pubkey object. If you provide a master_privkey, it is not necessary to
        provide a master_pubkey.
        master_privkey: Xtended_privkey object. If you provide a master_privkey, don't provide a _privkey.
        testnet: Boolean.
        segwit: Boolean.
        child_wallet: Boolean.
        """
        
        if m < 1 or m > 16 or n < 1 or n > 16: raise Exception("m and n must be between 1 and 16")
        if m > n: raise Exception("m must be always less or equal than n")   
        if len(public_key_list) != (n-1):
            msg = "n and the amount of public keys don't match. The amount of public keys must always be n-1."
            raise Exception(msg)
        
        self.index = None
        #getting the index of the private key in the list of public keys
        if _privkey is not None and master_privkey is None and master_pubkey is not None:
            self.wallet_type = "simple"
            self.master_privkey = None
            self.privkey = _privkey
            self.pubkey = PrivateKey(int.from_bytes(self.privkey,"big")).point.sec()
            for i,public_key in enumerate(public_key_list):
                if public_key == self.pubkey:
                    self.index = i
            if self.index is None: raise Exception ("Private key must be able to generate one of the public keys.")
            self.master_pubkey = master_pubkey
        
        elif _privkey is None and master_privkey is not None:
            self.privkey = _privkey
            self.wallet_type = "main"
            self.master_privkey = master_privkey
            self.master_pubkey = self.master_privkey.xtended_public_key
            
        elif _privkey is not None and master_privkey is not None:
            msg1 = "Provide either a Private key or a master_privkey, not both. Provide a _privkey if you are not "
            msg2 = "the main creator of the wallet or if you received this wallet as an invitation. Provide a "
            msg3 = "master_privkey if you have the public keys of the cosigners and you are in charge of sharing "
            msg4 = "this wallet with the cosigners."
            raise Exception (msg1+msg2+msg3+msg4)
            
        elif _privkey is None and master_privkey is None and master_pubkey is not None: 
            self.wallet_type = "watch-only"
            self.privkey = _privkey
            self.master_privkey = None
            self.master_pubkey = master_pubkey
        else:
            raise Exception ("You can only skip providing a master public key when you provide a master private key.")
        
        self.public_key_list = public_key_list
        self.m = m
        self.n = n
        self.addr_type = addr_type
        self.testnet = testnet
        self.segwit =segwit
        self.name = name
        self.parent_name = parent_name
        self.safe_index = safe_index
        
        #save the new wallet in the database
        #but first, lets convert the public keys to bytes to avoid problems in the database
        int_pubkeys = [int.from_bytes(x,"big") for x in self.public_key_list]
        if self.privkey is not None: int_privkey = int.from_bytes(self.privkey,"big")
        else: int_privkey = None
        if self.master_privkey is None: xpriv = None
        else: xpriv = str(self.master_privkey.xtended_key)
        
        self.start_conn()
        self.db.new_SHDSafeWallet(self.name, str(int_pubkeys), self.m, self.n, str(int_privkey),
                                  xpriv, str(self.master_pubkey.get_xtended_key()),
                                  self.addr_type, int(self.testnet), int(self.segwit), self.parent_name, 
                                  self.safe_index)
        
        self.close_conn()
        
    @classmethod    
    def from_privkey_masterpubkey(cls, name, public_key_list, master_pubkey, _privkey, m=2, n=6,addr_type="p2wsh", 
                                   testnet=False, segwit=True,parent_name=None,index=-1):
        """
        simple
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n,addr_type=addr_type,_privkey=_privkey, 
                   master_pubkey=master_pubkey,testnet=testnet, segwit=segwit,parent_name=parent_name, 
                   safe_index=index)
    
    @classmethod    
    def from_master_privkey(cls, name, public_key_list, master_privkey, m=2, n=6,addr_type="p2wsh",
                            testnet=False, segwit=True,  parent_name=None,index=-1):
        """
        main
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n, addr_type=addr_type, master_privkey=master_privkey, 
                   testnet=testnet, segwit=segwit,parent_name=parent_name, safe_index=index)
    
    @classmethod    
    def watch_only(cls, name, public_key_list,master_pubkey,m=2,n=6,addr_type="p2wsh",testnet=False, segwit=True, 
                    parent_name=None, index=-1):
        """
        watch-only
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n, addr_type=addr_type, master_pubkey=master_pubkey,
                   testnet=testnet, segwit=segwit, parent_name=parent_name, safe_index=index)
    
    @classmethod   
    def from_database(self, name):
        self.start_conn(self)
        w = self.db.recover_SHDSafeWallet(name)
        try: w = w[0]
        except: 
            print(f"wallet with name {name} not found ")
            return False
        self.close_conn(self)
        print(w)
        
        
        pubkeys = w[1][1:-2].split(", ")
        public_key_list = [int(x).to_bytes(33,"big") for x in pubkeys]
        print(public_key_list)
        if w[4] == "None": privkey = None
        else: privkey = int(w[4]).to_bytes(32,"big")
        if w[6] == "None": master_pubkey=None 
        else: master_pubkey = Xtended_pubkey.parse(w[6])
        if w[5] == "None": master_privkey = None
        else: master_privkey = Xtended_privkey.parse(w[5])
        if w[10] == "None": parent_name = None
        else: parent_name = w[10]
        
        return self(name, public_key_list, w[2],w[3],w[7],privkey,master_pubkey,master_privkey,
                    w[8],w[9],parent_name,w[11])
    
    def get_i(self,wallet_name, account_path, index):
        
        self.start_conn()

        last_index = self.db.get_max_index( account_path, wallet_name)

        if last_index is None: i = 0
        else: i = last_index + 1
        
        self.close_conn()
        return i
        
    def get_child_wallet(self, index, path):
        """
        This is the wallet for specific daily or weekly safes. Therefore, the index must comply with the format YYMMDD
        for daily safes, and YYWW for weekly safes.
        index: Integer.
        """
        #data validation
        if self.safe_index > -1: raise Exception ("Only master wallets can create child wallets.")  
        if index < 2000 or index > 999999 or (index > 9999 and index < 200101):
            raise Exception ("Bad index for daily or weekly safe. Follow YYMMDD for daily or YYWW for weekly.")
        
        full_path = path + str(index)
        #wallet creation depending on the type of wallet
        if self.wallet_type == "main":
            child_xtended_privkey = self.master_privkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.from_master_privkey( str(index)+"_"+self.name,self.public_key_list, child_xtended_privkey, 
                                                             self.m,self.n,self.addr_type,self.testnet, self.segwit,
                                                            self.name, index)
        elif self.wallet_type == "simple":
            child_xtended_pubkey = self.master_pubkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.from_privkey_masterpubkey(str(index)+"_"+self.name,self.public_key_list, child_xtended_pubkey,
                                                                   self.privkey, self.m, self.n,self.addr_type, 
                                                                   self.testnet, self.segwit,self.name, index)
        elif self.wallet_type == "watch-only":
            child_xtended_pubkey = self.master_pubkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.watch_only(str(index)+"_"+self.name,self.public_key_list,child_xtended_pubkey,self.m,self.n,
                                                    self.addr_type,self.testnet, self.segwit,self.name, index)
        
        return child_wallet
    
    def get_daily_safe_wallet(self, index=None):
        """
        Returns a daily_safe wallet based on the index.
        If today's wallet is desired, don't pass any argument or set index to None.
        """
        if self.safe_index >= 0: raise Exception ("Only master wallets can create safe wallets.")
        path = "m/"
        if index is None:
            today = datetime.datetime.now()
            index_string = str(today.year)[:2]+'{:02d}'.format(today.month)+'{:02d}'.format(today.day)
            index = int(index_string)
            
        safe_wallet = self.get_child_wallet(index, path)
        
        return safe_wallet
    
    
    def get_unused_addresses_list(self, change_addresses=False):
        
        unused_addresses = self.db.get_unused_safe_addresses(name = self.name)
        print(f"unused_addresses: {unused_addresses}")
        
        if    change_addresses: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==1]
        else: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==0]
        
        return unused_addresses_filtered
                               
                               
    def create_receiving_address(self, addr_type = "p2wsh",index=0):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        if addr_type.lower() ==    "p2sh":       _type  = P2SH
        elif addr_type.lower() ==   "p2wsh":     _type  = P2WSH
        elif addr_type.lower() == "p2sh_p2wsh":  _type  = P2SH_P2WSH
        else: raise Exception(f"{addr_type} is NOT a valid type of address.")
        receiving_path = "m/0/"
        #i = self.get_i(self.name,receiving_path, index)
        i = index
        path = receiving_path + str(i)
        print(f"Deposit address's Path: {path}")
        
        if self.privkey is not None: privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
        else: privkey = None
        
        account = SHMAccount(self.m, self.n, str(self.master_pubkey), self.public_key_list, privkey,
                             self.addr_type, self.testnet, self.segwit)
        
        address = account.get_deposit_address(i)
        self.start_conn()
        self.db.new_address(address,receiving_path,i,FALSE, _type, self.name, self.safe_index)
        self.close_conn()
        return address

    
    def create_change_address(self, addr_type = "p2wsh", index=None):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        receiving_path = "m/1/"
        
        i = self.get_i(self.name,receiving_path, index)
        path = receiving_path + str(i)
        print(f"Change address's Path: {path}")
        
        if self.privkey is not None: privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
        else: privkey = None
        
        account = SHMAccount(self.m, self.n, str(self.master_pubkey), self.public_key_list, privkey,
                             self.addr_type, self.testnet, self.segwit)
        
        address = account.get_change_address(i)
        self.start_conn()
        self.db.new_address(address,receiving_path,i,TRUE, P2WSH, self.name, self.safe_index)
        self.close_conn()
        return address
    
    def get_a_change_address(self):
        """
        Returns an Account object containing a change address. It returns an existing unused change
        address account, or creates a new one if necessary.
        """
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        self.start_conn()
        unused_addresses = self.get_unused_addresses_list(change_addresses=True)
        self.close_conn()
        
        if len(unused_addresses)>0: return unused_addresses[0][0]
        else: return self.create_change_address()
        
    def share(self):
        return {"public_key_list":self.public_key_list,"m":self.m, "n":self.n, "addr_type":self.addr_type,
                "master_pubkey":self.master_pubkey,"testnet":self.testnet, "segwit":self.segwit }
    

    def get_utxos(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        ################################ FOR DEVOLPMENT PORPUSES ONLY ###############################
        ############### CHANGE BACK TO ORIGINAL STATE ONCE DONE WITH TESTS ##########################
        #coins = self.db.look_for_coins(self.name) #ORIGINAL LINE!
        coins = self.db.look_for_coins("test1_master_wallet") #TEST LINE
        ############################################################################################
        self.close_conn()
        return coins
    
    def update_balance(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        addresses = self.db.get_all_addresses(self.name)
        self.close_conn()
        
        if self.testnet: coin_symbol = "btc-testnet"
        else: coin_symbol = "btc"
            
        for address in addresses:
            "address will be a touple with data (address,)"
            print(f"consulting blockchain for address {address}")
            addr_info = get_address_details(address[0], coin_symbol = coin_symbol, unspent_only=True)
            print(f"res for {address}:\n{addr_info}")
            
            if addr_info["unconfirmed_n_tx"] > 0:
                print("got unconfirmed transactions.")
                for utxo in addr_info["unconfirmed_txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 0):
                        print("new unconfirmed UTXO")
                        self.db.new_utxo(utxo["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],confirmed = 0)
            
            if addr_info["n_tx"] - addr_info["unconfirmed_n_tx"] > 0 :
                print("got transactions.")
                for utxo in addr_info["txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 1):
                        print("new confirmed UTXO")
                        self.db.new_utxo(addr_info["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],confirmed = 1)
      
        return self.get_balance()
        
    def get_coins(self, to_address_amount_list, segwit=True):
        send_all = False
        
        print(f"From wallet.send(): {to_address_amount_list}")
        #calculate total amount to send from the wallet
        total_amount = 0
        for output in to_address_amount_list:
            total_amount += output[1]
            
            
        #we validate that we have funds to carry out the transaction, or if we are trying to send all the funds  
        balance = self.get_balance()
        if total_amount>balance:
            raise Exception(f"Not enough funds in wallet for this transaction.\nOnly {balance} satoshis available")
        
        elif total_amount == balance: send_all = True
            
            
        #We choose the utxos to spend for the transaction
        all_utxos = self.get_utxos()
        
        #If we are trying to send all the funds, then we choose all the utxos
        if send_all: utxos = all_utxos
        #If not, then we have to choose in an efficient manner.
        else:    
            utxos = []
            utxo_total = 0
            
            #First we check if only one utxo can carry out the whole transaction
            for utxo in all_utxos:
                if utxo[2] > total_amount*1.1:
                    utxos = [utxo]
                    break
            #if we found that none of the utxos is big enough to carry out the whole tx, we have to choose several.
            if len(utxos)==0:    
                for utxo in all_utxos:
                    utxos.append(utxo)
                    utxo_total += utxo[2]
                    if utxo_total > (total_amount*1.1) : break
        
        return utxos, send_all
    
    def build_tx(self, utxos, to_address_amount_list, segwit=True, send_all = False):
        
        account = sefl.get_signing_account()
        
        #if we want to send all the funds we don't need a change address.
        if send_all: change_address = None    
        else:        change_address = self.get_a_change_address()
        
        #We create the Multi-Signature Transaction object
        tx_response = MultiSigTransaction.create(utxos,to_address_amount_list, account,change_address,None,True,send_all)
        
        return tx_response
    
    
     def get_signing_account(self):
        
        #we take care of validating and formatting the private key
        if self.privkey is not None: 
            privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
        else: raise Exception("Multi signature Transaction with a master key is not developed yet.")
        
        #We create the multisignature account to sign the transaction. 
        account = SHMAccount(self.m, self.n, str(self.master_pubkey), self.public_key_list, privkey,
                             self.addr_type, self.testnet, self.segwit)
        
        return account
        
        
    def broadcast_tx(self, tx):
        
        if self.testnet: symbol = "btc-testnet"
        else: symbol= "btc"

        env = Sqlite3Environment()
        BLOCKCYPHER_API_KEY = env.get_key("BLOCKCYPHER_API_KEY")[0][0]
        env.close_database()
        push = pushtx(tx_hex= tx.transaction.serialize().hex(), coin_symbol=symbol,api_key=BLOCKCYPHER_API_KEY)
        tx_id = tx.transaction.id()
        print(f"TRANSACTION ID: {tx_id}")

        #saving in db
        self.db.new_tx(tx_id, [ (x[0],x[1]) for x in utxos] ,
                       [str(x).split(":")+[i] for i,x in enumerate(tx.transaction.tx_outs)])

        #start new thread to check if the transaction goes through in the blockchain or not.
        #Maybe move this to the front end class to pop a warning when it doesn't go through
        mythread = threading.Thread(target=self.confirm_tx_sent,args=(tx_id,))
        mythread.start()

        return tx,push
        
        
    def send(self, to_address_amount_list, segwit=True):
        """
        to_address can be a single address or a list.
        amount can be an integer or a list of integers.
        If they are lists, they must be ordered in the same way. address 1 will e sent amount 1, 
        adrress 2 will be sent amount2.. adress n will be sent amount n.
        """
        #First let's get ready the inputs for the transaction
        utxos, send_all = self.get_coins(to_address_amount_list, segwit)
        
        #Now, let's build the transaction
        tx_response = self.buid_tx(utxos,to_address_amount_list, segwit, send_all)
        
        # tx_response will be a touple of the transaction object and a boolean (tx,READY) that tells us if the  
        #transaction is ready to be broadcasted or if it needs more signatures:
        if tx_response[1]: 
            #if it is ready, we broadcast the transaction
            self.broadcast_tx(tx_response[0])
            
        return tx_response 
        
    def sign_received_tx(self, tx):
        """
        tx: Tx object. The transaction object that needs to be signed.
        """
        
        #We create the multisignature account to sign the transaction.
        account = self.get_signing_account()
        
        tx_response = MultiSigTransaction.sign_received_tx(tx,account)
        
        if tx_response[1]: self.broadcast_tx(tx_response[0])
            
        return tx_response
            
            
            

In [7]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


public_keys = [ manager1.get_daily_safe_pubkey(), manager2.get_daily_safe_pubkey(), PrivateKey(int.from_bytes(b"Ashley Serna","little")).point.sec()]


words_master="other seek spend wise milk govern conduct crunch train wrestle twist cattle"
master_wallet = CorporateSuperWallet.recover_from_words(words_master,128,"",True)

8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799
7349618075688827342644292161792411154388397071333136974711210419399617419860687939173705085036785308619307965881391603570352784472532296452988784284606559


In [13]:
"""
main_wallet = SHDSafeWallet.from_master_privkey("test1_master_wallet",public_keys,
                                                master_privkey=master_wallet.get_child_from_path("m/44H/0H/1H"),
                                               n=4,testnet=True)
                                               """

connection with database made.
INSERT INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("test1_master_wallet","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"None","tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda","tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv","p2wsh",1,1,'None',-1) ;


In [8]:
main_wallet = SHDSafeWallet.from_database("test1_master_wallet")

connection with database made.
('test1_master_wallet', '[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]', 2, 4, 'None', 'tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda', 'tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv', 'p2wsh', 1, 1, 'None', -1)
[b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca', b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b"\x00M+B\x04\x93\x15\x14\xd4\xa4\xf7\x84]\xc6\x00\x92\x83L\x07\xff\xd1'\x8e\x04\x8e\x19Y\x9d\xe2\xa3K\xe4\x14"]
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,x

In [9]:
todays_wallet = main_wallet.get_daily_safe_wallet()

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("200825_test1_master_wallet","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 34904519403285236524485843581086005758991025139939414745136436266219764179988]",2,4,"None","tprv8iHAxPu9bvwprRqi3At2YUnF7hbeGbTYwfVBGa63McHTKqRfJGS1BNMKK68RCqmYyZPnZViDeKo1yVcPanhY5kyeWQrG1MXzaNmivL5MkoL","tpubDEyD6owPkJdVjtsVvpYcwtSMgj7aRveTWy5xZ68Lmt5rAKgRvfFbMryBVEgiykgQ1zJ9Ac65wncMebCYdEi5zkeMzPMwA2A7k1T7UVyAYAN","p2wsh",1,1,'test1_master_wallet',200825) ;


In [10]:
todays_address = todays_wallet.create_receiving_address()
todays_address

Deposit address's Path: m/0/0
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1ql4ajpghg3y93qxt66jk77hfva8an0s28pvjfurxy2vjzy6vgpq3s3eujg2", "m/0/", 0, 0, 1598341912, 5, "200825_test1_master_wallet", 200825) ;


'tb1ql4ajpghg3y93qxt66jk77hfva8an0s28pvjfurxy2vjzy6vgpq3s3eujg2'

In [11]:
todays_chnage = todays_wallet.create_change_address()

connection with database made.
Change address's Path: m/1/2
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qjwe2rufs4rx0aqvwz3dcg9s2gctuxs6k5ulth8psqfyk5m4cpresud3g2r", "m/1/", 2, 1, 1598307792, 5, "200824_test1_master_wallet", 200824) ;


In [12]:
todays_chnage

'tb1qjwe2rufs4rx0aqvwz3dcg9s2gctuxs6k5ulth8psqfyk5m4cpresud3g2r'

In [26]:
todays_wallet.get_a_change_address()

connection with database made.
SELECT address, change_addr, path, acc_index  
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = '200824_test1_master_wallet';
unused_addresses: [('tb1qakrn24j8e6alm7mnpft0ahj3qq6tjnfdnyvdn44ylelp9c8z62ds9pmh5n', 1, 'm/1/', 1), ('tb1qjwe2rufs4rx0aqvwz3dcg9s2gctuxs6k5ulth8psqfyk5m4cpresud3g2r', 1, 'm/1/', 2), ('tb1qx64e97ljs32k59gm3rcarwlseve9vsd3z6v97y835ahjrkps2hlq4vsrne', 0, 'm/0/', 0), ('tb1qyddvex3euj0qw67gla93vv82jl5d46af62acvlxql4w79dssnumskgs9ry', 1, 'm/1/', 0)]


'tb1qakrn24j8e6alm7mnpft0ahj3qq6tjnfdnyvdn44ylelp9c8z62ds9pmh5n'

In [27]:
todays_wallet.get_a_change_address()

connection with database made.
SELECT address, change_addr, path, acc_index  
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = '200824_test1_master_wallet';
unused_addresses: [('tb1qakrn24j8e6alm7mnpft0ahj3qq6tjnfdnyvdn44ylelp9c8z62ds9pmh5n', 1, 'm/1/', 1), ('tb1qjwe2rufs4rx0aqvwz3dcg9s2gctuxs6k5ulth8psqfyk5m4cpresud3g2r', 1, 'm/1/', 2), ('tb1qx64e97ljs32k59gm3rcarwlseve9vsd3z6v97y835ahjrkps2hlq4vsrne', 0, 'm/0/', 0), ('tb1qyddvex3euj0qw67gla93vv82jl5d46af62acvlxql4w79dssnumskgs9ry', 1, 'm/1/', 0)]


'tb1qakrn24j8e6alm7mnpft0ahj3qq6tjnfdnyvdn44ylelp9c8z62ds9pmh5n'

In [11]:
invite = main_wallet.share()
invite

{'public_key_list': [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca',
  b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}",
  b"\x00M+B\x04\x93\x15\x14\xd4\xa4\xf7\x84]\xc6\x00\x92\x83L\x07\xff\xd1'\x8e\x04\x8e\x19Y\x9d\xe2\xa3K\xe4\x14"],
 'm': 2,
 'n': 4,
 'addr_type': 'p2wsh',
 'master_pubkey': tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv,
 'testnet': 1,
 'segwit': 1}

In [12]:
manager_safe_wallet = manager1.accept_invite(invite,"manager1_daily_safe")

b'\xd3\xcdG\xa0\xa1\xbf\x86V\x879\xa2\x05Q3o\xb7(\x92Z\x19\xc8bzf\xf7\x9e\xd4\xfb\xc9\xf5\xbdP'
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("manager1_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 34904519403285236524485843581086005758991025139939414745136436266219764179988]",2,4,"95800709053964933282561433899801232546169819566735397106442210103505108450640","None","tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv","p2wsh",1,1,'None',-1) ;


In [13]:

todays_managers_wallet = manager_safe_wallet.get_daily_safe_wallet()

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("200825_manager1_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 34904519403285236524485843581086005758991025139939414745136436266219764179988]",2,4,"95800709053964933282561433899801232546169819566735397106442210103505108450640","None","tpubDEyD6owPkJdVjtsVvpYcwtSMgj7aRveTWy5xZ68Lmt5rAKgRvfFbMryBVEgiykgQ1zJ9Ac65wncMebCYdEi5zkeMzPMwA2A7k1T7UVyAYAN","p2wsh",1,1,'manager1_daily_safe',200825) ;


In [14]:
todays_managers_wallet.create_receiving_address()

Deposit address's Path: m/0/0
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1ql4ajpghg3y93qxt66jk77hfva8an0s28pvjfurxy2vjzy6vgpq3s3eujg2", "m/0/", 0, 0, 1598341951, 5, "200825_manager1_daily_safe", 200825) ;


'tb1ql4ajpghg3y93qxt66jk77hfva8an0s28pvjfurxy2vjzy6vgpq3s3eujg2'

In [32]:
todays_managers_wallet.get_a_change_address()

connection with database made.
SELECT address, change_addr, path, acc_index  
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = '200824_manager1_daily_safe';
unused_addresses: []
connection with database made.
Change address's Path: m/1/0
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qyddvex3euj0qw67gla93vv82jl5d46af62acvlxql4w79dssnumskgs9ry", "m/1/", 0, 1, 1598308076, 5, "200824_manager1_daily_safe", 200824) ;


'tb1qyddvex3euj0qw67gla93vv82jl5d46af62acvlxql4w79dssnumskgs9ry'